In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# %matplotlib widget
%config InlineBackend.figure_format = "svg"

import addict
import copy
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple

import celeri

plt.rcParams["text.usetex"] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files, create storage dictionaries, and do basic processing

In [43]:
command_file_name = "../data/western_north_america/basic_command.json"
command, segment, block, mesh_param, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
operators.meshes = [addict.Dict()] * len(meshes)
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this works correctly

# Get elastic operators and TDE smoothing operators

In [44]:
# Get all elastic operators for segments and TDEs
celeri.get_elastic_operators(operators, meshes, segment, station, command)

# Get TDE smoothing operators
celeri.get_all_mesh_smoothing_matrices(meshes, operators)
celeri.get_all_mesh_smoothing_matrices_simple(meshes, operators)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = './celeri_elastic_operators.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [55]:
import scipy

for i in range(len(meshes)):
    # Empty constraint matrix
    tri_slip_rate_constraints = np.zeros(
        (2 * meshes[i].n_tde, 2 * meshes[i].n_tde)
    )
    # Top constraints
    if meshes[i].edge_constraints[0] > 0:
        # Indices of top elements
        top_indices = np.asarray(np.where(meshes[i].top_elements))
        # Indices of top elements' 2 slip components
        top_idx = celeri.get_2component_index(top_indices)
        tri_slip_rate_constraints[top_idx, top_idx] = 1
    # Bottom constraints
    if meshes[i].edge_constraints[1] > 0:
        # Indices of bottom elements
        bot_indices = np.asarray(np.where(meshes[i].bot_elements))
        # Indices of bottom elements' 2 slip components
        bot_idx = celeri.get_2component_index(bot_indices)
        tri_slip_rate_constraints[bot_idx, bot_idx] = 1
    # Side constraints
    if meshes[i].edge_constraints[2] > 0:
        # Indices of side elements
        side_indices = np.asarray(np.where(meshes[i].side_elements))
        # Indices of side elements' 2 slip components
        side_idx = celeri.get_2component_index(side_indices)
        tri_slip_rate_constraints[side_idx, side_idx] = 1
    # Eliminate blank rows
    sum_constraint_columns = np.sum(tri_slip_rate_constraints, 1)
    tri_slip_rate_constraints = tri_slip_rate_constraints[sum_constraint_columns > 0, :]
    operators.meshes[i].tri_slip_rate_constraints = tri_slip_rate_constraints
    meshes[i].n_tde_constraints = np.sum(sum_constraint_columns > 0)
    


142


# Calculate non-elastic operators

In [ ]:
operators.rotation_to_velocities = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate_to_velocities, strain_rate_block_index = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_to_velocities = celeri.get_mogi_operator(mogi, station, command)

# Plot input data

In [ ]:
celeri.plot_input_summary(segment, station, block, meshes, mogi, sar, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)

# Sketching out the assembly of the block model system

$$
\begin{bmatrix}
    \mathrm{geodetic \; velocities} \\
    \mathrm{plate \; rotation \; constraints} \\
    \mathrm{slip \; rate \; constraints} \\
    \mathrm{TDE \; smoothing \; pseudodata = 0} \\
    \mathrm{TDE \; rate \; constraints} \\
    \mathrm{InSAR \; LOS \; changes} 
\end{bmatrix}
=
\begin{bmatrix}
    \mathrm{(rotations-elastic \; segments) \; to \; velocities} & \mathrm{TDEs \; to \; velocities} & \mathrm{block \; strain \; rate \; to \; velocities} & \mathrm{Mogi \; to \; velocities}\\
    \mathrm{identities}                                          & 0                                   & 0                                & 0\\
    \mathrm{plate \; rotations \; to \; slip \; rates}           & 0                                   & 0                                & 0\\
    0                                                            & \mathrm{TDE \; smoothing \; matrix} & 0                                & 0\\
    0                                                            & \mathrm{identities}                 & 0                                & 0\\
    \mathrm{(rotations-elastic \; segments) \; to \; LOS}        & \mathrm{TDEs \; to \; LOS}          & \mathrm{block \; strain \; rate \; to \; velocities}  & \mathrm{Mogi \; to \; LOS}
\end{bmatrix}
\begin{bmatrix}
    \mathrm{plate \; rotation \; rates} \\
    \mathrm{TDE \; slip \; rates} \\
    \mathrm{block \; strain \; rates} \\
    \mathrm{Mogi \; sources} 
\end{bmatrix}
$$

# Block model with block rotations, fully locked segments and partially locked subduction zone using the full tde_matrix and smoothing matrix

In [56]:
# TODO: What should this conversion be?
DEG_PER_MYR_TO_RAD_PER_YR = np.deg2rad(1) / 1e6
DEG_PER_MYR_TO_RAD_PER_YR = 1 / 1e6

# Create dictionary to store indices and sizes for operator building
index = addict.Dict()
index.n_stations = assembly.data.n_stations
index.vertical_velocities = np.arange(2, 3 * index.n_stations, 3)
index.n_blocks = len(block)
index.n_block_constraints = assembly.data.n_block_constraints
index.station_row_keep_index = celeri.get_keep_index_12(3 * len(station))
index.start_station_row = 0
index.end_station_row = 2 * len(station)
index.start_block_col = 0
index.end_block_col = 3 * len(block)
index.start_block_constraints_row = index.end_station_row
index.end_block_constraints_row = index.start_block_constraints_row + 3 * index.n_block_constraints
index.n_slip_rate_constraints = assembly.data.slip_rate_constraints.size
index.start_slip_rate_constraints_row = index.end_block_constraints_row
index.end_slip_rate_constraints_row = index.start_slip_rate_constraints_row + index.n_slip_rate_constraints

index.meshes = [addict.Dict()] * len(meshes)
index.n_tde_total = 0
for i in range(len(meshes)):
    index.meshes[i].n_tde = meshes[i].n_tde
    index.n_tde_total += index.meshes[i].n_tde
    index.meshes[i].n_tde_constraints = meshes[i].n_tde_constraints
    if i == 0:
        index.meshes[i].start_tde_col = index.end_block_col
        index.meshes[i].end_tde_col = index.meshes[i].start_tde_col + 2 * index.meshes[i].n_tde
        index.meshes[i].start_tde_smoothing_row = index.end_slip_rate_constraints_row
        index.meshes[i].end_tde_smoothing_row = index.meshes[i].start_tde_smoothing_row + 2 * index.meshes[i].n_tde
        index.meshes[i].start_tde_constraint_row = index.meshes[i].end_tde_smoothing_row
        index.meshes[i].end_tde_constraint_row = index.meshes[i].start_tde_constraint_row + 2 * index.meshes[i].n_tde_constraints
    else:
        index.meshes[i].start_tde_col = index.meshes[i - 1].end_tde_col
        index.meshes[i].end_tde_col = index.meshes[i].start_tde_col + 2 * index.meshes[i].n_tde
        index.meshes[i].start_tde_smoothing_row = index.meshes[i - 1].end_tde_smoothing_row
        index.meshes[i].end_tde_smoothing_row = index.meshes[i].start_tde_smoothing_row + 2 * index.meshes[i].n_tde
        index.meshes[i].start_tde_constraint_row = index.meshes[i - 1].end_tde_smoothing_row
        index.meshes[i].end_tde_constraint_row = index.meshes[i].start_tde_constraint_row + 2 * index.meshes[i].n_tde_constraints

index.start_tde_col = index.end_block_col
index.end_tde_col = index.start_tde_col + 2 * index.meshes[0].n_tde
index.start_tde_smoothing_row = index.end_slip_rate_constraints_row
index.end_tde_smoothing_row = index.start_tde_smoothing_row + 2 * index.meshes[0].n_tde

# Initialize data vector
estimation = addict.Dict()
estimation.data_vector = np.zeros(2 * index.n_stations + 3 * index.n_block_constraints + index.n_slip_rate_constraints + 2 * index.n_tde_total)

# Add GPS stations to data vector
estimation.data_vector[index.start_station_row:index.end_station_row] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel)

# Add block motion constraints to data vector
estimation.data_vector[index.start_block_constraints_row:index.end_block_constraints_row] = DEG_PER_MYR_TO_RAD_PER_YR * assembly.data.block_constraints

# Add slip rate constraints to data vector
estimation.data_vector[index.start_slip_rate_constraints_row:index.end_slip_rate_constraints_row] = assembly.data.slip_rate_constraints

# Initialize and build weighting matrix
estimation.weighting_vector = np.ones(2 * index.n_stations + 3 * index.n_block_constraints + index.n_slip_rate_constraints + 2 * index.n_tde_total)
estimation.weighting_vector[index.start_station_row:index.end_station_row] = celeri.interleave2(station.east_sig, station.north_sig)
estimation.weighting_vector[index.start_block_constraints_row:index.end_block_constraints_row] = 1.0
estimation.weighting_vector[index.start_slip_rate_constraints_row:index.end_slip_rate_constraints_row] = command.slip_constraint_weight * np.ones(index.n_slip_rate_constraints)

# Initialize linear operator
estimation.operator = np.zeros((2 * index.n_stations + 3 * index.n_block_constraints + index.n_slip_rate_constraints + 2 * index.n_tde_total,
                                3 * index.n_blocks + 2 * index.n_tde_total))

# Insert block rotations and elastic velocities from fully locked segments
operators.rotation_to_slip_rate_to_okada_to_velocities = operators.slip_rate_to_okada_to_velocities @ operators.rotation_to_slip_rate
estimation.operator[index.start_station_row:index.end_station_row, index.start_block_col:index.end_block_col] = operators.rotation_to_velocities[index.station_row_keep_index, :] - operators.rotation_to_slip_rate_to_okada_to_velocities[index.station_row_keep_index, :]

# Insert block motion constraints
estimation.operator[index.start_block_constraints_row:index.end_block_constraints_row, index.start_block_col:index.end_block_col] = operators.block_motion_constraints

# Insert slip rate constraints
estimation.operator[index.start_slip_rate_constraints_row:index.end_slip_rate_constraints_row, index.start_block_col:index.end_block_col] = operators.slip_rate_constraints

# Insert TDE to velocity matrix
for i in range(len(meshes)):
    # Insert TDE to velocity matrix
    tde_keep_row_index = celeri.get_keep_index_12(operators.meshes[i].tde_to_velocities.shape[0])
    tde_keep_col_index = celeri.get_keep_index_12(operators.meshes[i].tde_to_velocities.shape[1])
    estimation.operator[index.start_station_row:index.end_station_row, index.meshes[i].start_tde_col:index.meshes[i].end_tde_col] = operators.meshes[i].tde_to_velocities[tde_keep_row_index, :][:, tde_keep_col_index]

    # Insert TDE smoothing matrix
    smoothing_keep_index = celeri.get_keep_index_12(operators.meshes[i].smoothing_matrix.shape[0])
    estimation.operator[index.meshes[i].start_tde_smoothing_row:index.meshes[i].end_tde_smoothing_row, index.meshes[i].start_tde_col:index.meshes[i].end_tde_col] = meshes[i].smoothing_weight * operators.meshes[i].smoothing_matrix.toarray()[smoothing_keep_index, :][:, smoothing_keep_index]

# Solve the overdetermined linear system using only a weighting vector rather than matrix
estimation.state_covariance_matrix = np.linalg.inv(estimation.operator.T * estimation.weighting_vector @ estimation.operator)
estimation.state_vector = estimation.state_covariance_matrix @ estimation.operator.T * estimation.weighting_vector @ estimation.data_vector

celeri.post_process_estimation(estimation, operators, station, index)

TypeError: unsupported operand type(s) for *: 'int' and 'Dict'

# Plot model summary 

In [ ]:
celeri.plot_estimation_summary(segment, station, meshes, estimation, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)

In [ ]:
celeri.plot_matrix_abs_log(estimation.operator)

In [ ]:
plt.figure()
# ax = plt.gca()
celeri.plot_meshes(meshes, estimation.tde_strike_slip_rates, plt.gca())
plt.gca().set_aspect("equal", adjustable="box")
plt.show()